In [1]:
'''Code for fine-tuning Inception V3 for a new task.

Start with Inception V3 network, not including last fully connected layers.

Train a simple fully connected layer on top of these.


'''

import numpy as np
import pandas as pd
from keras import optimizers
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, Model
from keras.layers import Dense, Activation, Flatten, Dropout
import keras.applications.inception_v3 as inception
import final_models.cnn_d as OurNetwork
from sklearn.preprocessing import MultiLabelBinarizer
import matplotlib.pyplot as plt
IMSIZE = (196, 196)

PHOTO_PATH = r"\\devmachine\e$\data\katkam-secret-location\katkam-scaled"

def showImg(filename):
    img = image.load_img(PHOTO_PATH+"/"+filename, target_size=IMSIZE)
    x = image.img_to_array(img)
    plt.imshow(img)
    plt.show()

Using TensorFlow backend.


In [2]:
cleaned_data = pd.read_csv("cleaned_data.csv")
weather_array = list(map(lambda x:x.split(',') ,cleaned_data["Mapped"].values))
mlb = MultiLabelBinarizer()
Y = mlb.fit_transform(weather_array)
cleaned_data["Y"] = list(Y)

cleaned_data
print(mlb.classes_)

['Clear' 'Cloudy' 'Fog' 'Rain' 'Snow']


In [3]:
# cleaned_data[cleaned_data.apply(lambda x: x["Y"][2]==1,axis = 1)]

In [4]:
showImg("katkam-20161209130000.jpg")

In [5]:
#
def balance_data (df):
    clear_days_mask = df.apply(lambda x: x["Y"][0]==1,axis = 1)
    clear_days = df[clear_days_mask]
    non_clear_days = df[~clear_days_mask].sample(len(clear_days))
    final_data = clear_days.append(non_clear_days)
    final_data["Y2"] = cleaned_data.apply(lambda x: [x["Y"][0],(x["Y"][0]+1)%2],axis = 1)
    final_data = final_data.sample(frac=1)
    return final_data
final_data = balance_data(cleaned_data)

In [6]:
def sub_balanced_data(df):
    foggy_days  = cleaned_data[cleaned_data.apply(lambda x: x["Y"][2]==1,axis = 1)]
    rainy_days  = cleaned_data[cleaned_data.apply(lambda x: x["Y"][3]==1,axis = 1)]
    snowy_days  = cleaned_data[cleaned_data.apply(lambda x: x["Y"][4]==1,axis = 1)]
    just_cloudy_days  = cleaned_data[cleaned_data.apply(lambda x: (x["Y"][2]==0) and (x["Y"][3]==0) and (x["Y"][4]==0),axis = 1)]
    samples_per_label = len(snowy_days)
    final_data = foggy_days.sample(samples_per_label).append(rainy_days.sample(samples_per_label))
    final_data = final_data.append(snowy_days.sample(samples_per_label))
    final_data = final_data.append(just_cloudy_days.sample(samples_per_label))
    return final_data
# cleaned_data["Y3"] = cleaned_data.apply(lambda x: x["Y"],axis = 1)
cleaned_data["Y2"] = cleaned_data.apply(lambda x: [x["Y"][0],(x["Y"][0]+1)%2],axis = 1)
cleaned_data["Y3"] = cleaned_data.apply(lambda x: list(x["Y"])[2:5],axis = 1)
final_data = sub_balanced_data(cleaned_data)
train_test_mask = np.random.rand(len(final_data)) < 0.8
train_data = final_data[train_test_mask]
test_data = final_data[~train_test_mask]

In [7]:
base_model = OurNetwork.network(classes = 2)
for layer in base_model.layers:
    layer.trainable = False
x = Flatten(name='sub_flatten')(base_model.get_layer('block3_pool').output)
x = Dense(32, activation='relu', name='sub_fc1')(x)
x = Dropout(0.3)(x)
sub_predictions = Dense(3, activation='sigmoid', name='sub_predictions')(x)
predictions = (base_model.get_layer('predictions').output)
model = Model(inputs=base_model.input, outputs=[predictions, sub_predictions])
model.compile(loss='binary_crossentropy', optimizer="adam", loss_weights={'predictions': 0, 'sub_predictions': 1}, metrics=['accuracy'])
for layer in model.layers:
    layer.trainable = False
print(model.summary())

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, 196, 196, 3)   0                                            
____________________________________________________________________________________________________
block1_conv1 (Conv2D)            (None, 196, 196, 16)  448         input_1[0][0]                    
____________________________________________________________________________________________________
block1_conv2 (Conv2D)            (None, 196, 196, 16)  2320        block1_conv1[0][0]               
____________________________________________________________________________________________________
block1_pool (MaxPooling2D)       (None, 98, 98, 16)    0           block1_conv2[0][0]               
___________________________________________________________________________________________

In [8]:

#print(val_files)
# np.random.seed(0)
img_datagen = ImageDataGenerator(width_shift_range=0.2,height_shift_range=0.2)

def my_load_img(img_path,img_datagen,size):
    img = image.load_img(img_path, target_size=size)
    x = image.img_to_array(img)
    x = img_datagen.random_transform(x)
    x = img_datagen.standardize(x)
    return x
def my_img_generator(df,img_datagen,batch_size):
    index = 0
    count = 0
    img_datas=[]
    img_labels=[]
    img_sub_labels =[]
    while 1:
        # create numpy arrays of input data
        # and labels, from each line in the file
        item = df.iloc[index]
        if count < batch_size:
            img_datas.append(my_load_img(PHOTO_PATH+"/"+item["Filename"],img_datagen,IMSIZE))
#             [np.array(one_hot_labels),np.array(img_bboxes)]
            img_labels.append(item["Y2"])
            img_sub_labels.append(item["Y3"])

#             img_labels.append([np.array(item["Y2"]),np.array(item["Y3"])])

            index=(index+1)%df.shape[0]
            count+=1
        else:
            count=0
            yield (np.array(img_datas),[np.array(img_labels),np.array(img_sub_labels)])
                # yield (np.array(img_datas),np.array(img_bboxes))
#             else:
#                 yield (np.array(img_datas),np.array(one_hot_labels))
            img_datas = []
            img_labels = []
            img_sub_labels =[]
            img_bboxes=[]
#             random.shuffle(files)
            

batch_size=64

my_train_generator = my_img_generator(train_data,img_datagen,batch_size)
my_test_generator = my_img_generator(test_data,img_datagen,batch_size)


In [9]:
# Data training
# model.load_weights('final_models/sub_cnn.h5',by_name=True)
# model.save_weights('cnn_c.h5') 

# for i in range(0):
#     '''Refresh data'''
#     final_data = sub_balanced_data(cleaned_data)
#     train_test_mask = np.random.rand(len(final_data)) < 0.8
#     train_data = final_data[train_test_mask]
#     test_data = final_data[~train_test_mask]
#     my_train_generator = my_img_generator(train_data,img_datagen,batch_size)
#     my_test_generator = my_img_generator(test_data,img_datagen,batch_size)
    
#     model.fit_generator(
#             my_train_generator,
#             steps_per_epoch = 5,
#             epochs = 10,
#             validation_data = my_test_generator,
#             verbose = 2,
#             validation_steps = 5)
#     model.save_weights('new_sub_cnn.h5') 


In [10]:
img_path = PHOTO_PATH+"/"+"katkam-20161126060000.jpg"
model.load_weights('sub_cnn.h5') 

img = image.load_img(img_path , target_size=IMSIZE)
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)

# x = inception.preprocess_input(x)
# x = image.img_to_array(img)
# x = img_datagen.random_transform(x)
x = img_datagen.standardize(x)
preds = model.predict(x)
plt.imshow(img)
plt.show()
print('Predicted:', preds)
# [Clear, Cloudy, Fog, Rain, Snow]

Predicted: [array([[  9.63400907e-06,   9.99992490e-01]], dtype=float32), array([[ 0.62251705,  0.68031555,  0.19520335]], dtype=float32)]


In [11]:
pics = cleaned_data.loc[cleaned_data["Mapped"].str.contains("Fog")]

def listPhotots(photos):
    for p in photos:
        plt.figure(figsize=(8,8))
        img = image.load_img(PHOTO_PATH+"/"+ p, target_size=IMSIZE)
        plt.imshow(img)
        plt.show()
        print(p)
# listPhotots(cleaned_data[cleaned_data["Mapped"].str.contains("Snow")]["Filename"].values)

cleaned_data[(cleaned_data["Mapped"].str.contains("Snow"))]

Date/Time  Year  Month  Day Time_x                   Filename  \
1100  2016-12-05 06:00:00  2016     12    5  06:00  katkam-20161205060000.jpg   
1101  2016-12-05 07:00:00  2016     12    5  07:00  katkam-20161205070000.jpg   
1102  2016-12-05 08:00:00  2016     12    5  08:00  katkam-20161205080000.jpg   
1103  2016-12-05 09:00:00  2016     12    5  09:00  katkam-20161205090000.jpg   
1104  2016-12-05 10:00:00  2016     12    5  10:00  katkam-20161205100000.jpg   
1105  2016-12-05 11:00:00  2016     12    5  11:00  katkam-20161205110000.jpg   
1106  2016-12-05 12:00:00  2016     12    5  12:00  katkam-20161205120000.jpg   
1123  2016-12-09 07:00:00  2016     12    9  07:00  katkam-20161209070000.jpg   
1124  2016-12-09 08:00:00  2016     12    9  08:00  katkam-20161209080000.jpg   
1125  2016-12-09 09:00:00  2016     12    9  09:00  katkam-20161209090000.jpg   
1126  2016-12-09 10:00:00  2016     12    9  10:00  katkam-20161209100000.jpg   
1127  2016-12-09 12:00:00  2016     12    9  12:00  katkam-20161209120000.jpg   
1128  2016-12-09 13:00:00  2016     12    9  13:00  katkam-20161209130000.jpg   
1129  2016-12-09 14:00:00  2016     12    9  14:00  katkam-20161209140000.jpg   
1130  2016-12-09 15:00:00  2016     12    9  15:00  katkam-20161209150000.jpg   
1131  2016-12-09 16:00:00  2016     12    9  16:00  katkam-20161209160000.jpg   
1132  2016-12-09 17:00:00  2016     12    9  17:00  katkam-20161209170000.jpg   
1148  2016-12-12 06:00:00  2016     12   12  06:00  katkam-20161212060000.jpg   
1149  2016-12-12 07:00:00  2016     12   12  07:00  katkam-20161212070000.jpg   
1173  2016-12-18 06:00:00  2016     12   18  06:00  katkam-20161218060000.jpg   
1174  2016-12-18 07:00:00  2016     12   18  07:00  katkam-20161218070000.jpg   
1175  2016-12-18 08:00:00  2016     12   18  08:00  katkam-20161218080000.jpg   
1176  2016-12-18 09:00:00  2016     12   18  09:00  katkam-20161218090000.jpg   
1177  2016-12-18 10:00:00  2016     12   18  10:00  katkam-20161218100000.jpg   
1178  2016-12-18 11:00:00  2016     12   18  11:00  katkam-20161218110000.jpg   
1179  2016-12-18 12:00:00  2016     12   18  12:00  katkam-20161218120000.jpg   
1180  2016-12-18 13:00:00  2016     12   18  13:00  katkam-20161218130000.jpg   
1181  2016-12-18 14:00:00  2016     12   18  14:00  katkam-20161218140000.jpg   
1182  2016-12-18 15:00:00  2016     12   18  15:00  katkam-20161218150000.jpg   
1183  2016-12-18 16:00:00  2016     12   18  16:00  katkam-20161218160000.jpg   
...                   ...   ...    ...  ...    ...                        ...   
1391  2017-02-04 17:00:00  2017      2    4  17:00  katkam-20170204170000.jpg   
1392  2017-02-05 16:00:00  2017      2    5  16:00  katkam-20170205160000.jpg   
1393  2017-02-06 06:00:00  2017      2    6  06:00  katkam-20170206060000.jpg   
1394  2017-02-06 07:00:00  2017      2    6  07:00  katkam-20170206070000.jpg   
1395  2017-02-06 08:00:00  2017      2    6  08:00  katkam-20170206080000.jpg   
1398  2017-02-06 15:00:00  2017      2    6  15:00  katkam-20170206150000.jpg   
1399  2017-02-06 16:00:00  2017      2    6  16:00  katkam-20170206160000.jpg   
1400  2017-02-06 17:00:00  2017      2    6  17:00  katkam-20170206170000.jpg   
1401  2017-02-07 06:00:00  2017      2    7  06:00  katkam-20170207060000.jpg   
1408  2017-02-08 12:00:00  2017      2    8  12:00  katkam-20170208120000.jpg   
1410  2017-02-08 14:00:00  2017      2    8  14:00  katkam-20170208140000.jpg   
1411  2017-02-08 15:00:00  2017      2    8  15:00  katkam-20170208150000.jpg   
1412  2017-02-08 16:00:00  2017      2    8  16:00  katkam-20170208160000.jpg   
1413  2017-02-08 17:00:00  2017      2    8  17:00  katkam-20170208170000.jpg   
1512  2017-02-27 08:00:00  2017      2   27  08:00  katkam-20170227080000.jpg   
1513  2017-02-27 09:00:00  2017      2   27  09:00  katkam-20170227090000.jpg   
1514  2017-02-27 10:00:00  2017      2   27  10:00  katkam-20170227100000.jpg   
1517  2017-02-28 07:00:00  2017      2   28  07

In [12]:
def batchPredict(imageNames):
    result = []
    for filename in imageNames:
        img_path = PHOTO_PATH+"/"+filename
        img = image.load_img(img_path , target_size=IMSIZE)
        x = image.img_to_array(img)
        x = np.expand_dims(x, axis=0)
        x = img_datagen.standardize(x)
        preds = model.predict(x)
        result.append([preds[0].reshape(2),preds[1].reshape(3)])
    return result

def get_Y3_Label(preds):
    y3_label = ""
    if(preds[0][0]>0.6):
        return "N/A"
    else:
        if(preds[1][0]>0.5):
            y3_label+="Fog"
        if(preds[1][1]>0.5):
            y3_label+=" Rain"
        if(preds[1][2]>0.5):
            y3_label+=" Snow"
        if(y3_label == ""):
            y3_label="Cloudy"
    return y3_label
cleaned_data["Predict_combined"] = batchPredict(cleaned_data["Filename"].values)
cleaned_data["Predict_Y2_Label"] = cleaned_data["Predict_combined"].apply(lambda x: "Clear" if x[0][0]>0.6 else "Non_Clear")
cleaned_data["Predict_Y3_Label"] = cleaned_data["Predict_combined"].apply(lambda x: get_Y3_Label(x))


In [13]:
cleaned_data.sample(10)

Date/Time  Year  Month  Day Time_x                   Filename  \
991   2016-11-19 13:00:00  2016     11   19  13:00  katkam-20161119130000.jpg   
1353  2017-01-30 07:00:00  2017      1   30  07:00  katkam-20170130070000.jpg   
463   2016-08-31 06:00:00  2016      8   31  06:00  katkam-20160831060000.jpg   
138   2016-06-28 07:00:00  2016      6   28  07:00  katkam-20160628070000.jpg   
808   2016-10-23 07:00:00  2016     10   23  07:00  katkam-20161023070000.jpg   
1236  2016-12-27 12:00:00  2016     12   27  12:00  katkam-20161227120000.jpg   
1546  2017-03-04 16:00:00  2017      3    4  16:00  katkam-20170304160000.jpg   
971   2016-11-15 15:00:00  2016     11   15  15:00  katkam-20161115150000.jpg   
271   2016-07-22 13:00:00  2016      7   22  13:00  katkam-20160722130000.jpg   
1863  2017-04-19 10:00:00  2017      4   19  10:00  katkam-20170419100000.jpg   

           Mapped        Weather                Y      Y2         Y3  \
991        Cloudy         Cloudy  [0, 1, 0, 0, 0]  [0, 1]  [0, 0, 0]   
1353       Cloudy  Mostly Cloudy  [0, 1, 0, 0, 0]  [0, 1]  [0, 0, 0]   
463   Rain,Cloudy   Rain Showers  [0, 1, 0, 1, 0]  [0, 1]  [0, 1, 0]   
138         Clear          Clear  [1, 0, 0, 0, 0]  [1, 0]  [0, 0, 0]   
808   Rain,Cloudy           Rain  [0, 1, 0, 1, 0]  [0, 1]  [0, 1, 0]   
1236  Rain,Cloudy   Rain Showers  [0, 1, 0, 1, 0]  [0, 1]  [0, 1, 0]   
1546       Cloudy  Mostly Cloudy  [0, 1, 0, 0, 0]  [0, 1]  [0, 0, 0]   
971   Rain,Cloudy           Rain  [0, 1, 0, 1, 0]  [0, 1]  [0, 1, 0]   
271         Clear   Mainly Clear  [1, 0, 0, 0, 0]  [1, 0]  [0, 0, 0]   
1863  Rain,Cloudy           Rain  [0, 1, 0, 1, 0]  [0, 1]  [0, 1, 0]   

                                       Predict_combined Predict_Y2_Label  \
991   [[4.20904e-09, 1.0], [0.00802029, 0.348565, 0....        Non_Clear   
1353  [[7.58818e-16, 1.0], [0.431633, 0.532099, 0.26...        Non_Clear   
463   [[1.8284e-06, 0.999998], [0.348256, 0.850716, ...        Non_Clear   
138   [[0.933269, 0.0662978], [0.0239417, 0.00797589...            Clear   
808   [[2.45559e-19, 1.0], [0.443439, 0.867656, 0.03...        Non_Clear   
1236  [[0.000134951, 0.999864], [0.182609, 0.854727,...        Non_Clear   
1546  [[0.70777, 0.292808], [0.000113904, 0.00236944...            Clear   
971   [[9.8909e-09, 1.0], [0.0482147, 0.412107, 0.00...        Non_Clear   
271   [[0.906854, 0.0927748], [6.0204e-05, 0.0261964...            Clear   
1863  [[0.00117759, 0.998815], [0.0222105, 0.517397,...        Non_Clear   

     Predict_Y3_Label  
991            Cloudy  
1353             Rain  
463              Rain  
138               N/A  
808              Rain  
1236             Rain  
1546              N/A  
971            Cloudy  
271               N/A  
1863             Rain

In [14]:
# batchPredict(cleaned_data.sample(1)["Filename"].values)[0][1]